# Simple RAG에서의 Contextual Chunk Headers (CCH)

RAG는 응답을 생성하기 전에 **외부의 관련 지식**을 검색하여, 언어 모델의 사실 기반 정확도를 높이는 방식입니다. 하지만 일반적인 chunking 방식은 중요한 문맥 정보가 사라지는 경우가 많아, 검색 정확도에 한계가 생길 수 있습니다.

이를 보완하기 위해 **Contextual Chunk Headers(CCH)** 기법을 도입하였습니다. 이 방식은 각 chunk 앞에 **문서 제목이나 섹션 제목과 같은 상위 문맥 정보**를 붙여서 임베딩합니다. 이렇게 하면 검색 품질이 향상되며, 문맥에서 벗어난 잘못된 응답도 줄일 수 있습니다.

#### 이 노트북의 주요 단계는 다음과 같습니다:

1. **데이터 수집(Data Ingestion)**: 텍스트 데이터를 불러오고 전처리합니다.
2. **문맥 헤더 기반 Chunking**: 섹션 제목 등을 추출하여 각 chunk 앞에 붙입니다.
3. **임베딩 생성(Embedding Creation)**: 문맥이 보강된 chunk들을 숫자 벡터로 변환합니다.
4. **의미 기반 검색(Semantic Search)**: 사용자 질의와 관련된 chunk들을 검색합니다.
5. **응답 생성(Response Generation)**: 검색된 텍스트를 바탕으로 언어 모델이 응답을 생성합니다.
6. **응답 평가(Evaluation)**: 별도의 평가 기준을 활용해 응답의 정확도를 점검합니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 효율적 검색을 위한 문단별 요약 헤더 생성
검색 효율을 높이기 위해, 각 chunk마다 LLM 모델을 사용해 내용을 잘 설명하는 헤더를 생성합니다.

In [ ]:
def generate_chunk_header(chunk, model_name="gpt-4o-mini"):
    """
    주어진 텍스트 청크에 대한 간결한 제목을 생성하는 LLM 모델을 활용합니다.

    Args:
    chunk (str): 제목으로 요약할 텍스트 청크
    model (str): 헤더를 생성하는데 사용할 모델. 

    Returns:
    str: 생성된 헤더/제목.
    """
    # 시스템 프롬프트
    system_prompt = "주어진 텍스트에 대한 제목을 생성합니다. 제목은 최대 10자 이내로 생성합니다."
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )

    # 생성된 헤더/제목 반환, 앞/뒤 공백 제거
    return response.choices[0].message.content.strip()

In [ ]:
def chunk_text_with_headers(text, n, overlap):
    """
    Chunk text를 작은 청크로 나누고, 각 청크에 대해 LLM 모델을 활용해 헤더를 생성합니다.

    Args:
    text (str): 청크할 텍스트
    n (int): 각 청크의 문자 수
    overlap (int): 청크 간 중복 문자 수

    Returns:
    List[dict]: 헤더와 텍스트가 포함된 딕셔너리 리스트
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트

    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # 텍스트 청크 추출
        header = generate_chunk_header(chunk)  # LLM 모델을 활용해 헤더 생성
        chunks.append({"header": header, "text": chunk})  # 헤더와 청크를 리스트에 추가

    return chunks  

In [ ]:
# 청크 생성
# 추출한 텍스트를 1000자씩, 200자가 겹치도록 나눕니다.
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# 첫 번째 청크 출력
print("Sample Chunk:")
print("Header:", text_chunks[0]['header'])
print("Content:", text_chunks[0]['text'])

## 분할된 청크와 해당 청크의 header의 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

In [ ]:
from tqdm import tqdm

embeddings = [] 

for chunk in tqdm(text_chunks, desc="Generating embeddings"):
    # 텍스트 청크의 임베딩 생성
    text_embedding = create_embeddings(embedding_model, [chunk["text"]], device=device, batch_size=1)
    # 헤더 청크의 임베딩 생성
    header_embedding = create_embeddings(embedding_model, [chunk["header"]], device=device, batch_size=1)
    
    embeddings.append({"header": chunk["header"], "text": chunk["text"], "embedding": text_embedding, "header_embedding": header_embedding})

## Semantic Search
코사인 유사도를 구현하여 사용자 쿼리에 가장 관련성이 높은 텍스트 청크를 찾습니다.

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def semantic_search(query, chunks, k=5):
    """
    쿼리에 대한 가장 관련성 높은 청크를 검색합니다.

    Args:
    query (str): 사용자 쿼리.
    chunks (List[dict]): 임베딩이 포함된 텍스트 청크 리스트.
    k (int): 상위 k개 결과.

    Returns:
    List[dict]: 상위 k개 관련성 높은 청크.
    """
    # 쿼리에 대한 임베딩을 생성
    query_embedding = create_embeddings(embedding_model, [query])[0]

    similarities = []  
    
    # 각 청크에 대해 유사도 점수를 계산
    for chunk in chunks:
        # 쿼리 임베딩과 청크 텍스트 임베딩 간의 코사인 유사도 계산
        sim_text = cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"][0]))
        # 쿼리 임베딩과 청크 헤더 임베딩 간의 코사인 유사도 계산
        sim_header = cosine_similarity(np.array(query_embedding), np.array(chunk["header_embedding"][0]))
        # 평균 유사도 점수 계산
        avg_similarity = (sim_text + sim_header) / 2
        # 청크와 평균 유사도 점수를 리스트에 추가
        similarities.append((chunk, avg_similarity))

    # 유사도 점수에 따라 청크를 내림차순으로 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)
    # 상위 k개 관련성 높은 청크 반환
    return [x[0] for x in similarities[:k]]

## 테스트

In [ ]:
import pandas as pd

# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 시맨틱 검색 수행
top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

# 결과 출력
print("Query:", query)
for i, chunk in enumerate(top_chunks):
    print(f"Header {i+1}: {chunk['header']}")
    print(f"Content:\n{chunk['text']}\n")

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(system_prompt, user_message ,model_name='gpt-4.1-nano'):
    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_message}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 사용자 프롬프트
user_prompt = "\n".join([f"Header: {chunk['header']}\nContent:\n{chunk['text']}" for chunk in top_chunks])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# 시스템 프롬프트
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

# 응답 생성
ai_response = generate_response(system_prompt, user_prompt,'gpt-4.1-nano')

In [ ]:
print(ai_response.choices[0].message.content)

## 생성 응답 평가하기
생성된 답변과 예상 답변을 비교하여 평가합니다. 답변 평가시에는 LLM을 사용합니다.

In [ ]:
# 평가 시스템 프롬프트
evaluate_system_prompt = """You are an intelligent evaluation system. 
Assess the AI assistant's response based on the provided context. 
- Assign a score of 1 if the response is very close to the true answer. 
- Assign a score of 0.5 if the response is partially correct. 
- Assign a score of 0 if the response is incorrect.
Return only the score (0, 0.5, or 1)."""

# 참고(정답) 답변
true_answer = df['generation_gt'][0]

# 평가 프롬프트
evaluation_prompt = f"""
User Query: {query}
AI Response: {ai_response}
True Answer: {true_answer}
{evaluate_system_prompt}
"""

# 평가
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt, 'gpt-4.1-mini')

# 평가 점수 출력
print("Evaluation Score:", evaluation_response.choices[0].message.content)

## 전체 추론 및 평가

In [ ]:
result = []

for i in range(len(df)):
    query = df['query'][i]
    top_chunks = semantic_search(query, embeddings, k=3)

        
    # 사용자 프롬프트
    user_prompt = "\n".join([f"Header: {chunk['header']}\nContent:\n{chunk['text']}" for chunk in top_chunks])
    user_prompt = f"{user_prompt}\nQuestion: {query}"
    

    # 시스템 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
    
    # 응답 생성
    ai_response = generate_response(system_prompt, user_prompt,'gpt-4.1-nano')

    print(ai_response.choices[0].message.content)
    
    # 평가 프롬프트
    evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][i]}\n{evaluate_system_prompt}"

    # 평가 응답 생성
    evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt, 'gpt-4.1-mini')

    # 평가 응답 출력
    print(evaluation_response.choices[0].message.content)
    result.append(evaluation_response.choices[0].message.content)


In [ ]:
## 점수 계산

In [ ]:
import numpy as np
result = [float(x.replace('\n','').replace('Score: ',''))for x in result]
np.average(result)